<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/11_summary_writing_translation.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 11. Summary, Writing, Translation

## Overview  
In this exercise, we will use the Solar API for translation and summarization. By applying the LLM Chain techniques we have previously learned, we can generate summaries in the desired format and translate English to Korean using the Solar Translation API. This tutorial will guide you through the process of combining summarization and translation effectively.

## Purpose of the Exercise
The purpose of this exercise is to attain proficiency in utilizing the LLM Chain and modify output formats by applying prompt engineering techniques. By the end of this tutorial, users will be able to create summaries and translate text seamlessly, improving their ability to manage multilingual content using the Solar API.


In [1]:
! pip3 install -qU langchain-upstage  requests python-dotenv

In [2]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [3]:
solar_text = """
SOLAR 10.7B: Scaling Large Language Models with Simple yet Effective Depth Up-Scaling

We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
demonstrating superior performance in various natural language processing (NLP) tasks.
Inspired by recent efforts to efficiently up-scale LLMs,
we present a method for scaling LLMs called depth up-scaling (DUS),
which encompasses depthwise scaling and continued pretraining.
In contrast to other LLM up-scaling methods that use mixture-of-experts,
DUS does not require complex changes to train and inference efficiently.
We show experimentally that DUS is simple yet effective
in scaling up high-performance LLMs from small ones.
Building on the DUS model, we additionally present SOLAR 10.7B-Instruct,
a variant fine-tuned for instruction-following capabilities,
surpassing Mixtral-8x7B-Instruct.
SOLAR 10.7B is publicly available under the Apache 2.0 license,
promoting broad access and application in the LLM field.
"""

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage(model="solar-pro")

prompt_template = PromptTemplate.from_template(
    """
    Please do {action} on the following text:
    ---
    TEXT: {text}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [5]:
chain.invoke({"action": "three line summarize", "text": solar_text})

'SOLAR 10.7B is a 10.7B parameter LLM with improved NLP performance, scaled using a simple yet effective depth up-scaling (DUS) method. DUS involves depthwise scaling and continued pretraining, without the need for complex changes like mixture-of-experts. SOLAR 10.7B-Instruct, a fine-tuned variant for instruction-following, outperforms Mixtral-8x7B-Instruct. The model is publicly accessible under the Apache 2.0 license.'

In [7]:
chain.invoke(
    {
        "action": "Please rewrite this so that children and grandma can understand it easily.",
        "text": solar_text,
    }
)

'We\'re excited to share about our new "SOLAR 10.7B" toy! It\'s a big, smart toy with 10.7 billion tiny parts, and it can do many cool things with words and sentences. \n\nWe made SOLAR 10.7B by using a special trick called "depth up-scaling." This trick helps us make our smart toy even better without making it too complicated. \n\nOur smart toy is different from other big toys because it doesn\'t need a special helper called "mixture-of-experts" to work well. \n\nWe also made a special version of SOLAR 10.7B called "SOLAR 10.7B-Instruct." This version is great at following instructions and is even better than another popular toy called "Mixtral-8x7B-Instruct."\n\nWe want everyone to have fun with SOLAR 10.7B, so we\'re sharing it for free! You can use it for many word and sentence projects, and it\'s available under the "Apache 2.0 license."'

In [8]:
from langchain_core.prompts import ChatPromptTemplate

enko_translation = ChatUpstage(model="solar-1-mini-translate-enko")

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{text}"),
    ]
)

chain = chat_prompt | enko_translation | StrOutputParser()

In [9]:
chain.invoke(
    {
        "text": """We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters,
demonstrating superior performance in various natural language processing (NLP) tasks. """
    }
)

'107억 개의 매개변수를 가진 대규모 언어 모델(LLM)인 SOLAR 10.7B를 소개합니다. \n다양한 자연어 처리(NLP) 작업에서 뛰어난 성능을 보여줍니다.'

In [10]:
style_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "we present a method for scaling LLMs called depth up-scaling (DUS)"),
        ("ai", "DUS라는 방법에 대해 알려줄께. DUS는 LLMs를 확장하는 방법이야."),
        (
            "human",
            "In contrast to other LLM up-scaling methods that use mixture-of-experts, DUS does not require complex changes to train and inference efficiently.",
        ),
        (
            "ai",
            "다른 LLM 확장 방법과는 달리, DUS는 복잡한 변경 사항 없이 효율적으로 훈련하고 추론할 수 있어.",
        ),
        ("human", "{text}"),
    ]
)

chain = style_prompt | enko_translation | StrOutputParser()

In [11]:
chain.invoke(
    {
        "text": """We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, demonstrating superior performance in various natural language processing (NLP) tasks. """
    }
)

'우리는 SOLAR 10.7B라는 대형 언어 모델(LLM)을 소개하고, 다양한 자연어 처리(NLP) 작업에서 우수한 성능을 보여줘.'